In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from actual_stats_calc import get_actual_stats

df = pd.read_csv('tkrb_stats_test.csv')

# UI
sword_dd = widgets.Dropdown(options = sorted(df['Name'].unique()), description = 'Sword:')
level_is = widgets.IntSlider(value = 1, min = 1, max = 99, description = 'Level:')
is_max = widgets.Checkbox(value = True, description = 'Max Stats', disabled = False, indent = False)
output_area = widgets.Output()

def refresh_ui(change):
    with output_area:
        clear_output()
        try:
            name = sword_dd.value
            level = level_is.value
            character = df[df['Name'] == name].iloc[0] # used to check sword type
            TOKU_LEVEL = {
                'Tantou':20,
                'Wakizashi':20,
                'Uchigatana':20, # 2 flower lv20, 3 flower lv25, since Uchi has both then default 20
                'Tachi':25,
                'Ootachi':25,
                'Yari':25,
                'Naginata':25,
                'Tsurugi':25,
            }

            # automatically set status based on level
            if 'Kiwame' in name:
                status = 'Kiwame'
            elif level >= TOKU_LEVEL.get(character['Type']):
                status = 'Toku'
            else:
                status = 'Base'

            # check for if user wants max stats
            if is_max.value and status != 'Base': status = f'{status}_Max'
            # get target info
            character = df[(df['Name'] == name) & (df['Status'] == status)]

            if character.empty:
                print(f"Could not find {name}'s {status} form.")
            else:
                actual_stats = get_actual_stats(character.iloc[0], level)

            #testing
            print(f"--- {name} ({status}) LV.{level} ---")
            for key, val in actual_stats.items():
                if key not in ['Name', 'Status', 'Type']:
                    print(f"{key}: {val}")
        except Exception as e:
            print(f"D; something happened: {e}")


def update_level_limit(change):
    if 'Kiwame' in sword_dd.value:
        level_is.max = 199
    else:
        level_is.max = 99

# observers
sword_dd.observe(refresh_ui, names = 'value')
sword_dd.observe(update_level_limit, names = 'value')
level_is.observe(refresh_ui, names = 'value')
is_max.observe(refresh_ui, names = 'value')
        
# layout
ui_layout = widgets.VBox([
    widgets.HBox([sword_dd, is_max]), # sword selection
    level_is, # level selection
    output_area, #display output
])

display(ui_layout)
refresh_ui(None)